In [1]:
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI
from reasoning_engines.langchain_llm_wrappers import QuickOpenAIClient
from reasoning_engines.langchain_llm_wrappers import QuickAzureOpenAIClient
openai_client = QuickOpenAIClient()
azure_openai_client = QuickAzureOpenAIClient('1106')
llm = QuickAzureChatOpenAI('1106')
# openai_response = openai_client.chat_completions_create(query='How is the weather in Seoul, Bangkok and LA?', tools=openai_functions)
# azure_openai_client.chat_completions_create(query='hello')
from tools.get_tools import DefaultSchemasTools
default_schemas_and_tools = DefaultSchemasTools(azure_gpt_version='1106')
schemas_and_tools = default_schemas_and_tools.schemas_and_tools()
schemas = default_schemas_and_tools.schemas()
tools = default_schemas_and_tools.tools()
tool_dictionary = default_schemas_and_tools.tool_dictionary()
from langchain.utils.openai_functions import convert_pydantic_to_openai_tool
openai_functions = [convert_pydantic_to_openai_tool(x) for x in schemas]


from utils.wrappers import retry
from openai import RateLimitError
@retry(allowed_exceptions=(RateLimitError,))
def get_function_response(function_name, function_args):
    return tool_dictionary[function_name].run(**function_args)


from agents.parallel_func_calling_agent import OpenAIParallelFuntionCallingAgent
from chains.qa_evaluators import CustomQAEvaluator

parallel_calling_agent = OpenAIParallelFuntionCallingAgent(
    reasoninig_engine = QuickAzureChatOpenAI('1106'),
    base_prompt = 'jet-taekyo-lee/parallel-function-calling-agent',
    schemas_and_tools = DefaultSchemasTools(azure_gpt_version='1106'),
    evaluator = CustomQAEvaluator(llm=QuickAzureChatOpenAI('1106')),
    action_word = 'Tool invocations in parallel',
    use_chat_completion_api = True,
    azure_apenai_client = QuickAzureOpenAIClient('1106'),
)

query = 'How is the weather in Seoul, Bangkok and LA? Plus How old are the current Korean president and Joe Biden?'
query = 'How is the weather in Seoul, Bangkok and LA tomorrow?'
hello_query = 'Hello'


msg = parallel_calling_agent.agent_step(query)
# msg 


[{'role': 'system', 'content': 'You are a helpful assistant. Respond to the human as helpfully and accurately as possible in the same language as the human. But your intermediate processes should be done in English for more decent results. Answer in a consise manner with only a few words or a sentence if possible.'}, {'role': 'user', 'content': 'How is the weather in Seoul, Bangkok and LA tomorrow?'}]


[INFO] Tool invocations in parallel (step 0): [Tool 0-> OpenWeatherMap(IANA_timezone=Asia/Seoul, Days_from_now=1), Tool 1-> OpenWeatherMap(IANA_timezone=Asia/Bangkok, Days_from_now=1), Tool 2-> OpenWeatherMap(IANA_timezone=America/Los_Angeles, Days_from_now=1)]
[INFO] Results in parallel (step 0): [Tool 1-> timezone: Seoul, current time(year-month-day): 2023-11-30, temperature: -2.1299999999999955, humidity: 36, wind_speed: 5.93, weather_summary: Expect a day of partly cloudy with clear spells., Tool 2-> timezone: Bangkok, current time(year-month-day): 2023-11-30, temperature: 31.78000000000003, humidity: 42, wind_speed: 3.65, weather_summary: There will be partly cloudy today., Tool 3-> timezone: Los Angeles, current time(year-month-day): 2023-11-29, temperature: 17.189999999999998, humidity: 43, wind_speed: 4.91, weather_summary: Expect a day of partly cloudy with clear spells.]


In [ ]:
parallel_calling_agent.tool_dictionary

In [ ]:
import json
parallel_calling_agent.tool_dictionary['OpenWeatherMap'].run(**json.loads('{"IANA_timezone": "Asia/Seoul", "Days_from_now": 1}'))

In [ ]:
args = {'IANA_timezone': 'Asia/Bangkok'}

In [ ]:
name_dict = {tool.__name__: tool for tool in parallel_calling_agent.tools}
name_dict

In [ ]:
parallel_calling_agent.tools[0].name

In [ ]:
from 